In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder



In [2]:
df=pd.read_csv('final_data_for_modelling.csv')
df_locations=pd.read_csv('data/locations.csv')
rating_df=pd.read_csv('data/rating_processed.csv')

df=df.merge(df_locations,how='left',left_on=['neighbourhood'],right_on=['locations'])
df=df.merge(rating_df,how='left',on=['lat','lon'])

enc = OrdinalEncoder()
ohe    = OneHotEncoder(sparse=False)

enc.fit(df[["gender"]])
df['rating'].fillna(df['rating'].mode()[0], inplace=True)
df[["gender"]] = enc.transform(df[["gender"]])
encoded_columns =    ohe.fit_transform(df[['appointment_day_of_week']])
data_hot_encoded = pd.DataFrame(encoded_columns, index=df.index)
data_hot_encoded.columns = ohe.get_feature_names(['appointment_day_of_week'])

data_other_cols = df.drop(columns='appointment_day_of_week')

#Concatenate the two dataframes :
df = pd.concat([data_hot_encoded, data_other_cols], axis=1)

df.head()
df.to_csv('test.csv')



/opt/homebrew/Caskroom/miniconda/base/envs/aipi510project/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [3]:
y=df['showed']
X=df.drop('showed', axis=1)
X=X[['gender','age','scholarship','hypertension','diabetes','alcoholism','handicap','sms_received','days_between_appointment_and_scheduled_day','tempmax','temp','feelslikemax','feelslike','humidity','windspeed','solarradiation','solarenergy','uvindex','appointment_day_of_week_Friday','appointment_day_of_week_Monday','appointment_day_of_week_Tuesday','appointment_day_of_week_Wednesday','appointment_day_of_week_Thursday','appointment_day_of_week_Friday','appointment_day_of_week_Saturday','rating']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [4]:
model=RandomForestClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

print(f'accuracy for test scores: {accuracy_score(y_test,y_pred):3f}')


accuracy for test scores: 0.784679
